In [1]:
import os

import torch
import imageio.v3 as iio
import csv
import numpy as np
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets
from matplotlib import pyplot as plt
from torchvision import transforms
import torch.nn.functional as F
import datetime

In [2]:
data_path = 'C:/Users/Igor/igye/projects/python/d2l2/data/cifar10/'
cifar10 = datasets.CIFAR10(data_path, train=True, download=True,   transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4915, 0.4823, 0.4468),
                         (0.2470, 0.2435, 0.2616))
]))
cifar10_val = datasets.CIFAR10(data_path, train=False, download=True,   transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4915, 0.4823, 0.4468),
                         (0.2470, 0.2435, 0.2616))
]))

Files already downloaded and verified
Files already downloaded and verified


In [3]:
label_map = {0: 0, 2: 1}
class_names = ['airplane', 'bird']
cifar2 = [(img, label_map[label])
          for img, label in cifar10
          if label in [0, 2]]
cifar2_val = [(img, label_map[label])
              for img, label in cifar10_val
              if label in [0, 2]]

In [4]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(8 * 8 * 8, 32)
        self.fc2 = nn.Linear(32, 2)
    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = out.view(-1, 8 * 8 * 8)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

In [12]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1): #2
        loss_train = 0.0
        for imgs, labels in train_loader: #3
            outputs = model(imgs) #4
            loss = loss_fn(outputs, labels) #5

            l2_lambda = 0.001
            l2_norm = sum(p.pow(2.0).sum() for p in model.parameters())
            loss = loss + l2_lambda * l2_norm
            
            optimizer.zero_grad() #6
            loss.backward() #7
            optimizer.step() #8
            loss_train += loss.item() #9
        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader))) #10

In [6]:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64,
                                           shuffle=True) #1
model = Net() # #2
optimizer = optim.SGD(model.parameters(), lr=1e-2) # #3
# #4
training_loop(  # 5
    n_epochs=100,
    optimizer=optimizer,
    model=model,
    loss_fn=(nn.CrossEntropyLoss()),
    train_loader=train_loader,
)

2024-06-30 10:34:40.966121 Epoch 1, Training loss 0.580411914427569
2024-06-30 10:34:56.574983 Epoch 10, Training loss 0.3301950957934568
2024-06-30 10:35:16.216046 Epoch 20, Training loss 0.28892021839785725
2024-06-30 10:35:36.352520 Epoch 30, Training loss 0.26423808059115317
2024-06-30 10:35:58.114175 Epoch 40, Training loss 0.24528778425067854
2024-06-30 10:36:20.941486 Epoch 50, Training loss 0.22959943585524892
2024-06-30 10:36:43.314174 Epoch 60, Training loss 0.21262002930899335
2024-06-30 10:37:06.945112 Epoch 70, Training loss 0.19489082357116566
2024-06-30 10:37:31.155918 Epoch 80, Training loss 0.18118039352498996
2024-06-30 10:37:55.773904 Epoch 90, Training loss 0.17061165743952345
2024-06-30 10:38:22.517224 Epoch 100, Training loss 0.15239373831802114


In [7]:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64, shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=64, shuffle=False)
def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0
        with torch.no_grad(): #1
            for imgs, labels in loader:
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1) #2
                total += labels.shape[0] #3
                correct += int((predicted == labels).sum()) #4
        print("Accuracy {}: {:.2f}".format(name , correct / total))
validate(model, train_loader, val_loader)

Accuracy train: 0.94
Accuracy val: 0.89


In [8]:
numel_list = [p.numel()
              for p in model.parameters()
              if p.requires_grad == True]
sum(numel_list), numel_list

(18090, [432, 16, 1152, 8, 16384, 32, 64, 2])

In [9]:
torch.save(model.state_dict(), data_path + 'birds_vs_airplanes.pt')

In [10]:
loaded_model = Net()
loaded_model.load_state_dict(torch.load(data_path + 'birds_vs_airplanes.pt'))

<All keys matched successfully>

In [13]:
optimizer = optim.SGD(loaded_model.parameters(), lr=1e-2) # #3
training_loop( #5
    n_epochs = 30,
    optimizer = optimizer,
    model = loaded_model,
    loss_fn = nn.CrossEntropyLoss(),
    train_loader = train_loader,
)

2024-06-30 11:34:32.707331 Epoch 1, Training loss 0.19355731020877315
2024-06-30 11:34:49.985115 Epoch 10, Training loss 0.18582125743673106
2024-06-30 11:35:11.685179 Epoch 20, Training loss 0.17812238766509256
2024-06-30 11:35:35.214690 Epoch 30, Training loss 0.1710864312614605


In [17]:
a = torch.rand(100)*1e3
print(f'{a.min()=}')
print(f'{a.max()=}')
a_mean = a.mean()
a_std = a.std()
print(f'{a_mean=}')
print(f'{a_std=}')
b = (a - a_mean) / a_std
print(f'{b.min()=}')
print(f'{b.max()=}')
b_mean = b.mean()
b_std = b.std()
print(f'{b_mean=}')
print(f'{b_std=}')

a.min()=tensor(27.5252)
a.max()=tensor(991.1556)
a_mean=tensor(526.6780)
a_std=tensor(284.8911)
b.min()=tensor(-1.7521)
b.max()=tensor(1.6304)
b_mean=tensor(2.3127e-07)
b_std=tensor(1.)
